In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Load and split the text file
with open("general.txt", encoding="utf8") as f:
    raw_text = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)
text_chunks = text_splitter.split_text(raw_text)

# Create embeddings and vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
vector_store.save_local("faiss_index")


In [9]:
def get_conversational_chain():
    """Create a conversational chain for answering questions."""
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details,

    Context:\n {context}\n
    Question: \n{question}\n
    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Load a question-answering chain with the specified model and prompt
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

def answer_question(user_question):
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    response = chain(
        {"input_documents": docs, "question": user_question}, return_only_outputs=True
    )

    return response["output_text"]


In [10]:
print("Chat with the context loaded from 'general.txt'")
while True:
    user_question = input("Ask a question (or type 'exit' to quit): ")
    if user_question.lower() == 'exit':
        print("Exiting. Goodbye!")
        break
    
    answer = answer_question(user_question)
    print(f"\nAnswer: {answer}\n")


Chat with the context loaded from 'general.txt'

Answer: The provided context does not contain any information about "hi". Therefore, I cannot answer this question from the provided context.


Answer: **Srishti - The Literary Club**

* Provides a platform for students to showcase their literary aspirations and talents.
* Hosts events such as poetry-reading sessions, debates, extempore speeches, quizzes, cross-word games, book and film reviews, play enactments, and classical movie screenings.
* Organizes events in regional languages like Tamil, Malayalam, Telugu, and Hindi.

**Natyasudha - The Dance Club**

* Focuses on dance performances and training.

**Ragasudha - The Music Club**

* Focuses on music performances and training.

**Amrita Talkies**

* Aims to provide members with an experience and technical knowledge on film appreciation and filmmaking.
* Conducts movie screenings, workshops, and competitions.

**Sahaya Club**

* A social club that aims to help the needy, including chi

In [11]:
answer_question("When did the Coimbatore campus start ?")

'1994'

In [20]:
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, Request
from pydantic import BaseModel
from typing import List, Dict, Any
from langchain.vectorstores import FAISS


conversation_context: Dict[str, Dict[str, List[Dict[str, Any]]]] = {}

class QueryRequest(BaseModel):
    session_id: str
    input_text: str

app = FastAPI()

@app.get("/")
def index():
    return {"message": "Hello! Use the /get-response endpoint to chat."}

@app.post("/get-response/")
def get_response(request: QueryRequest):
    """
    Handles a conversational API endpoint.
    """
    session_id = request.session_id
    user_input = request.input_text

    # Retrieve or initialize the context for the session
    if session_id not in conversation_context:
        conversation_context[session_id] = {"history": []}

    # Add user input to the conversation history
    conversation_history = conversation_context[session_id]["history"]
    conversation_history.append({"user": user_input})

    # Generate a response using the conversational chain
    response = answer_question(user_input)

    # Add the response to the conversation history
    conversation_history.append({"bot": response})

    # Save the updated history
    conversation_context[session_id]["history"] = conversation_history

    return {"session_id": session_id, "response": response, "history": conversation_history}

# CORS setup
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Main server setup
if __name__ == "__main__":
    config = uvicorn.Config(app, host="127.0.0.1", port=8000, log_level="info")
    server = uvicorn.Server(config)
    asyncio.run(server.serve())


INFO:     Started server process [14008]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63340 - "POST /get-response HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:63340 - "POST /get-response/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63386 - "POST /get-response HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:63389 - "POST /get-response HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:63386 - "POST /get-response/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63390 - "POST /get-response/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63488 - "POST /get-response HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:63488 - "POST /get-response/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63491 - "POST /get-response HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:63492 - "POST /get-response/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63700 - "POST /get-response HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:63700 - "POST /get-response/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63702 - "OPTIONS /get-response HTTP/1.1" 200 OK
INFO:     127.0.0.1:63703 - "POST /get-response HT

In [18]:
import nest_asyncio
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI

nest_asyncio.apply()

app = FastAPI()

@app.get("/")
def index():
    return {"hello": "world"}

@app.get("/get-response/{input_text}")
def get_student(input_text: str):
    output = answer_question(input_text)
    return {"response": f"Mocked response for: {output}"}

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Use this block for Jupyter or environments with an active event loop
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [14008]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nithi\anaconda3\envs\College\Li

INFO:     127.0.0.1:62647 - "GET /get-response/When%20did%20the%20Coimbatore%20campus%20start HTTP/1.1" 200 OK
INFO:     127.0.0.1:62664 - "GET /get-response/what%20are%20the%20clubs%20at%20amrita HTTP/1.1" 200 OK
INFO:     127.0.0.1:62669 - "GET /explain%20them HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14008]


## **FASTAPI**

In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
!ngrok config add-authtoken 2byhqoCrlLHUUIkJkBxspq8hs1R_6cPvKVeHHsTTuPwE3WTiw

                                                                                                    
Installing ngrok ... 


Traceback (most recent call last):
  File "C:\Users\nithi\anaconda3\envs\college\Lib\site-packages\pyngrok\installer.py", line 125, in install_ngrok
    _install_ngrok_zip(ngrok_path, download_path)
  File "C:\Users\nithi\anaconda3\envs\college\Lib\site-packages\pyngrok\installer.py", line 140, in _install_ngrok_zip
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nithi\anaconda3\envs\college\Lib\zipfile\__init__.py", line 1331, in __init__
    self.fp = io.open(file, filemode)
              ^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 22] Invalid argument: 'C:\\Users\\nithi\\AppData\\Local\\Temp\\ngrok-v3-stable-windows-amd64.zip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\nithi\anaconda3\envs\College\Scripts\ngrok.exe\__main__.py", line 7, in <modul

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [2]:
app = FastAPI()

@app.get("/")
def index():
    return {'hello' : 'world'}

@app.get('/get-response/{input_text}')
def get_student(input_text : str):
    return generate_response(input_text)

In [3]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

PyngrokNgrokInstallError: An error occurred while downloading ngrok from https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-windows-amd64.zip: [Errno 22] Invalid argument: 'C:\\Users\\nithi\\AppData\\Local\\Temp\\ngrok-v3-stable-windows-amd64.zip'